In [45]:
import requests
import datetime
import pprint
import sqlite3
import json
import pandas as pd

In [46]:
last_week = (datetime.date.today() - datetime.timedelta(days=7)).strftime("%Y-%m-%d")
link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={last_week}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
conn = sqlite3.connect('NASA_NEO.db')

Exploration of ETL

In [47]:
# for date, neo_list in neo_data.items():
#     for neo in neo_list:
#         cursor.execute('''
#                     INSERT INTO neo VALUES (?, ?, ?, ?, ?, ?)
#                     ''', (
#                         neo['id'],
#                         neo['name'],
#                         neo['is_potentially_hazardous_asteroid'],
#                         neo['is_sentry_object'],
#                         neo['close_approach_data'],
#                         neo['estimated_diameter']
#                     ))
# conn.commit()

ELT

In [48]:
def load_data():
    response = requests.get(link)
    data = response.json()
    neo_data = data['near_earth_objects']
  
    #pprint.pprint(neo_data)

    cursor = conn.cursor()

    cursor.executescript('''

             CREATE TABLE IF NOT EXISTS neo (
                            date_added datetime,
                            data json
            )
        ''')

    for date, neo_list in neo_data.items():
        for neo in neo_list:
            cursor.execute('INSERT INTO neo VALUES (?, ?)',
                            (datetime.datetime.today(), json.dumps(neo)))
    conn.commit()

load_data()

Exploration of SQL transformation

In [49]:
# df = pd.read_sql_query('''
#                        select json_extract(data, "$.name") as name,
#                        json_extract(data, "$.id") as ID,
#                        json_extract(data, "$.is_potentially_hazardous_asteroid") as potentially_hazardous
#                        from neo;
#                        ''', conn)

# df["potentially_hazardous"] = df["potentially_hazardous"].astype(bool)

Pandas transformation

In [50]:
df = pd.read_sql_query('''SELECT * FROM neo''', conn)

df['data'] = df['data'].apply(json.loads)

json_df = pd.json_normalize(df['data'])

json_df

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,2329437,2329437,329437 (2002 OA22),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2329437,19.41,True,"[{'close_approach_date': '2023-09-26', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/2329437?ap...,0.348783,0.779902,348.782733,779.901901,0.216723,0.484608,1144.300342,2558.733352
1,3177195,3177195,(2004 EO20),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3177195,22.50,False,"[{'close_approach_date': '2023-09-26', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3177195?ap...,0.084053,0.187949,84.053340,187.948982,0.052228,0.116786,275.765561,616.630540
2,3459484,3459484,(2009 JM1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3459484,23.20,False,"[{'close_approach_date': '2023-09-26', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3459484?ap...,0.060891,0.136157,60.891262,136.157002,0.037836,0.084604,199.774489,446.709337
3,3670290,3670290,(2014 HJ184),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3670290,21.65,False,"[{'close_approach_date': '2023-09-26', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3670290?ap...,0.124324,0.277997,124.324001,277.996916,0.077251,0.172739,407.887154,912.063403
4,3843427,3843427,(2019 OV3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3843427,24.26,False,"[{'close_approach_date': '2023-09-26', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3843427?ap...,0.037373,0.083568,37.372743,83.567994,0.023222,0.051927,122.613991,274.173218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,3843536,3843536,(2019 PM2),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3843536,21.83,False,"[{'close_approach_date': '2023-09-30', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3843536?ap...,0.114434,0.255882,114.433973,255.882143,0.071106,0.158998,375.439556,839.508369
184,3989291,3989291,(2020 BD13),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3989291,23.30,False,"[{'close_approach_date': '2023-09-30', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/3989291?ap...,0.058151,0.130029,58.150704,130.028927,0.036133,0.080796,190.783156,426.604105
185,54051300,54051300,(2020 QG3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54051300,21.32,True,"[{'close_approach_date': '2023-09-30', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54051300?a...,0.144729,0.323623,144.728805,323.623447,0.089930,0.201090,474.832053,1061.756748
186,54115824,54115824,(2021 CN5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54115824,27.80,False,"[{'close_approach_date': '2023-09-30', 'close_...",False,http://api.nasa.gov/neo/rest/v1/neo/54115824?a...,0.007321,0.016370,7.320740,16.369672,0.004549,0.010172,24.018176,53.706275


Extracting nested close approach data

In [51]:
json_df['close_approach_data'][0]

[{'close_approach_date': '2023-09-26',
  'close_approach_date_full': '2023-Sep-26 15:59',
  'epoch_date_close_approach': 1695743940000,
  'relative_velocity': {'kilometers_per_second': '15.7156596389',
   'kilometers_per_hour': '56576.3746998676',
   'miles_per_hour': '35154.3590461483'},
  'miss_distance': {'astronomical': '0.2974523576',
   'lunar': '115.7089671064',
   'kilometers': '44498239.123438312',
   'miles': '27649923.6473082256'},
  'orbiting_body': 'Earth'}]

Converting list of dict to dict

In [52]:
for n in json_df.index.values:
        json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}

json_df

C:\Users\Jack\AppData\Local\Temp\ipykernel_6880\523775224.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k:v for element in json_df['close_approach_data'][n] for k,v in element.items()}


,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,estimated_diameter.kilometers.estimated_diameter_max,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,estimated_diameter.miles.estimated_diameter_min,estimated_diameter.miles.estimated_diameter_max,estimated_diameter.feet.estimated_diameter_min,estimated_diameter.feet.estimated_diameter_max
0,2329437,2329437,329437 (2002 OA22),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2329437,19.41,True,"{'close_approach_date': '2023-09-26', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/2329437?ap...,0.348783,0.779902,348.782733,779.901901,0.216723,0.484608,1144.300342,2558.733352
1,3177195,3177195,(2004 EO20),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3177195,22.50,False,"{'close_approach_date': '2023-09-26', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3177195?ap...,0.084053,0.187949,84.053340,187.948982,0.052228,0.116786,275.765561,616.630540
2,3459484,3459484,(2009 JM1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3459484,23.20,False,"{'close_approach_date': '2023-09-26', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3459484?ap...,0.060891,0.136157,60.891262,136.157002,0.037836,0.084604,199.774489,446.709337
3,3670290,3670290,(2014 HJ184),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3670290,21.65,False,"{'close_approach_date': '2023-09-26', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3670290?ap...,0.124324,0.277997,124.324001,277.996916,0.077251,0.172739,407.887154,912.063403
4,3843427,3843427,(2019 OV3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3843427,24.26,False,"{'close_approach_date': '2023-09-26', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3843427?ap...,0.037373,0.083568,37.372743,83.567994,0.023222,0.051927,122.613991,274.173218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,3843536,3843536,(2019 PM2),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3843536,21.83,False,"{'close_approach_date': '2023-09-30', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3843536?ap...,0.114434,0.255882,114.433973,255.882143,0.071106,0.158998,375.439556,839.508369
184,3989291,3989291,(2020 BD13),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3989291,23.30,False,"{'close_approach_date': '2023-09-30', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3989291?ap...,0.058151,0.130029,58.150704,130.028927,0.036133,0.080796,190.783156,426.604105
185,54051300,54051300,(2020 QG3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54051300,21.32,True,"{'close_approach_date': '2023-09-30', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54051300?a...,0.144729,0.323623,144.728805,323.623447,0.089930,0.201090,474.832053,1061.756748
186,54115824,54115824,(2021 CN5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54115824,27.80,False,"{'close_approach_date': '2023-09-30', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54115824?a...,0.007321,0.016370,7.320740,16.369672,0.004549,0.010172,24.018176,53.706275


In [53]:
cad_df = pd.json_normalize(json_df['close_approach_data'])

cad_df

,close_approach_date,close_approach_date_full,epoch_date_close_approach,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,relative_velocity.miles_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,miss_distance.miles
0,2023-09-26,2023-Sep-26 15:59,1695743940000,Earth,15.7156596389,56576.3746998676,35154.3590461483,0.2974523576,115.7089671064,44498239.123438312,27649923.6473082256
1,2023-09-26,2023-Sep-26 09:47,1695721620000,Earth,3.7493697142,13497.730971158,8386.9651137176,0.2326105867,90.4855182263,34798048.309770329,21622504.5708313802
2,2023-09-26,2023-Sep-26 20:54,1695761640000,Earth,18.9668256714,68280.5724171152,42426.8923444014,0.2440373587,94.9305325343,36507469.061945969,22684689.3720696122
3,2023-09-26,2023-Sep-26 02:59,1695697140000,Earth,19.5415017367,70349.4062519443,43712.3852347151,0.4265898628,165.9434566292,63816934.838472236,39654004.5280034168
4,2023-09-26,2023-Sep-26 04:14,1695701640000,Earth,10.2016957272,36726.104618078,22820.1731722719,0.3800277452,147.8307928828,56851341.222822724,35325785.3574286312
...,...,...,...,...,...,...,...,...,...,...,...
183,2023-09-30,2023-Sep-30 10:56,1696071360000,Earth,13.887658663,49995.571186647,31065.3036631696,0.4228055303,164.4713512867,63250806.757100461,39302228.8502939218
184,2023-09-30,2023-Sep-30 06:22,1696054920000,Earth,29.6233023399,106643.8884235018,66264.3650040364,0.4551151295,177.0397853755,68084253.977974165,42305593.685466877
185,2023-09-30,2023-Sep-30 07:07,1696057620000,Earth,13.0569466774,47005.0080385244,29207.0840226043,0.0572520013,22.2710285057,8564777.447717231,5321905.9259545478
186,2023-09-30,2023-Sep-30 22:03,1696111380000,Earth,19.726973613,71017.105006692,44127.2672748526,0.1384929905,53.8737733045,20718256.388730235,12873727.557277843


In [54]:
df = pd.concat([json_df, cad_df, df], axis=1)

df

,id,neo_reference_id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,close_approach_data,is_sentry_object,links.self,estimated_diameter.kilometers.estimated_diameter_min,...,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,relative_velocity.miles_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,miss_distance.miles,date_added,data
0,2329437,2329437,329437 (2002 OA22),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2329437,19.41,True,"{'close_approach_date': '2023-09-26', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/2329437?ap...,0.348783,...,Earth,15.7156596389,56576.3746998676,35154.3590461483,0.2974523576,115.7089671064,44498239.123438312,27649923.6473082256,2023-10-04 16:38:22.332234,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,3177195,3177195,(2004 EO20),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3177195,22.50,False,"{'close_approach_date': '2023-09-26', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3177195?ap...,0.084053,...,Earth,3.7493697142,13497.730971158,8386.9651137176,0.2326105867,90.4855182263,34798048.309770329,21622504.5708313802,2023-10-04 16:38:22.333232,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3459484,3459484,(2009 JM1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3459484,23.20,False,"{'close_approach_date': '2023-09-26', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3459484?ap...,0.060891,...,Earth,18.9668256714,68280.5724171152,42426.8923444014,0.2440373587,94.9305325343,36507469.061945969,22684689.3720696122,2023-10-04 16:38:22.333232,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3670290,3670290,(2014 HJ184),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3670290,21.65,False,"{'close_approach_date': '2023-09-26', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3670290?ap...,0.124324,...,Earth,19.5415017367,70349.4062519443,43712.3852347151,0.4265898628,165.9434566292,63816934.838472236,39654004.5280034168,2023-10-04 16:38:22.333232,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3843427,3843427,(2019 OV3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3843427,24.26,False,"{'close_approach_date': '2023-09-26', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3843427?ap...,0.037373,...,Earth,10.2016957272,36726.104618078,22820.1731722719,0.3800277452,147.8307928828,56851341.222822724,35325785.3574286312,2023-10-04 16:38:22.333232,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,3843536,3843536,(2019 PM2),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3843536,21.83,False,"{'close_approach_date': '2023-09-30', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3843536?ap...,0.114434,...,Earth,13.887658663,49995.571186647,31065.3036631696,0.4228055303,164.4713512867,63250806.757100461,39302228.8502939218,2023-10-04 16:38:27.648992,{'links': {'self': 'http://api.nasa.gov/neo/re...
184,3989291,3989291,(2020 BD13),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3989291,23.30,False,"{'close_approach_date': '2023-09-30', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/3989291?ap...,0.058151,...,Earth,29.6233023399,106643.8884235018,66264.3650040364,0.4551151295,177.0397853755,68084253.977974165,42305593.685466877,2023-10-04 16:38:27.648992,{'links': {'self': 'http://api.nasa.gov/neo/re...
185,54051300,54051300,(2020 QG3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54051300,21.32,True,"{'close_approach_date': '2023-09-30', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54051300?a...,0.144729,...,Earth,13.0569466774,47005.0080385244,29207.0840226043,0.0572520013,22.2710285057,8564777.447717231,5321905.9259545478,2023-10-04 16:38:27.648992,{'links': {'self': 'http://api.nasa.gov/neo/re...
186,54115824,54115824,(2021 CN5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54115824,27.80,False,"{'close_approach_date': '2023-09-30', 'close_a...",False,http://api.nasa.gov/neo/rest/v1/neo/54115824?a...,0.007321,...,Earth,19.72

Cleaning up df

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188 entries, 0 to 187
Data columns (total 30 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   id                                                    188 non-null    object 
 1   neo_reference_id                                      188 non-null    object 
 2   name                                                  188 non-null    object 
 3   nasa_jpl_url                                          188 non-null    object 
 4   absolute_magnitude_h                                  188 non-null    float64
 5   is_potentially_hazardous_asteroid                     188 non-null    bool   
 6   close_approach_data                                   188 non-null    object 
 7   is_sentry_object                                      188 non-null    bool   
 8   links.self                                            188 no

In [56]:
df = df.drop(['neo_reference_id', 'close_approach_data', 'links.self',
        'estimated_diameter.kilometers.estimated_diameter_min',
        'estimated_diameter.kilometers.estimated_diameter_max',
        'estimated_diameter.miles.estimated_diameter_min',
        'estimated_diameter.miles.estimated_diameter_max',
        'estimated_diameter.feet.estimated_diameter_min',
        'estimated_diameter.feet.estimated_diameter_max',
        'close_approach_date', 'epoch_date_close_approach',
        'relative_velocity.miles_per_hour',
        'miss_distance.miles'], axis=1
        )

df

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter.meters.estimated_diameter_min,estimated_diameter.meters.estimated_diameter_max,close_approach_date_full,orbiting_body,relative_velocity.kilometers_per_second,relative_velocity.kilometers_per_hour,miss_distance.astronomical,miss_distance.lunar,miss_distance.kilometers,date_added,data
0,2329437,329437 (2002 OA22),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2329437,19.41,True,False,348.782733,779.901901,2023-Sep-26 15:59,Earth,15.7156596389,56576.3746998676,0.2974523576,115.7089671064,44498239.123438312,2023-10-04 16:38:22.332234,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,3177195,(2004 EO20),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3177195,22.50,False,False,84.053340,187.948982,2023-Sep-26 09:47,Earth,3.7493697142,13497.730971158,0.2326105867,90.4855182263,34798048.309770329,2023-10-04 16:38:22.333232,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3459484,(2009 JM1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3459484,23.20,False,False,60.891262,136.157002,2023-Sep-26 20:54,Earth,18.9668256714,68280.5724171152,0.2440373587,94.9305325343,36507469.061945969,2023-10-04 16:38:22.333232,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3670290,(2014 HJ184),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3670290,21.65,False,False,124.324001,277.996916,2023-Sep-26 02:59,Earth,19.5415017367,70349.4062519443,0.4265898628,165.9434566292,63816934.838472236,2023-10-04 16:38:22.333232,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3843427,(2019 OV3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3843427,24.26,False,False,37.372743,83.567994,2023-Sep-26 04:14,Earth,10.2016957272,36726.104618078,0.3800277452,147.8307928828,56851341.222822724,2023-10-04 16:38:22.333232,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,3843536,(2019 PM2),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3843536,21.83,False,False,114.433973,255.882143,2023-Sep-30 10:56,Earth,13.887658663,49995.571186647,0.4228055303,164.4713512867,63250806.757100461,2023-10-04 16:38:27.648992,{'links': {'self': 'http://api.nasa.gov/neo/re...
184,3989291,(2020 BD13),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3989291,23.30,False,False,58.150704,130.028927,2023-Sep-30 06:22,Earth,29.6233023399,106643.8884235018,0.4551151295,177.0397853755,68084253.977974165,2023-10-04 16:38:27.648992,{'links': {'self': 'http://api.nasa.gov/neo/re...
185,54051300,(2020 QG3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54051300,21.32,True,False,144.728805,323.623447,2023-Sep-30 07:07,Earth,13.0569466774,47005.0080385244,0.0572520013,22.2710285057,8564777.447717231,2023-10-04 16:38:27.648992,{'links': {'self': 'http://api.nasa.gov/neo/re...
186,54115824,(2021 CN5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54115824,27.80,False,False,7.320740,16.369672,2023-Sep-30 22:03,Earth,19.726973613,71017.105006692,0.1384929905,53.8737733045,20718256.388730235,2023-10-04 16:38:27.648992,{'links': {'self': 'http://api.nasa.gov/neo/re...


In [57]:
df = df.rename(columns={
    'estimated_diameter.meters.estimated_diameter_min': 'estimated_diameter_meters_min',
    'estimated_diameter.meters.estimated_diameter_max': 'estimated_diameter_meters_max',
    'relative_velocity.kilometers_per_second': 'relative_velocity_kilometers_per_second',
    'relative_velocity.kilometers_per_hour': 'relative_velocity_kilometers_per_hour',
    'miss_distance.astronomical': 'miss_distance_astronomical',
    'miss_distance.lunar': 'miss_distance_lunar',
    'miss_distance.kilometers': 'miss_distance_kilometers',
    })
df

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_meters_min,estimated_diameter_meters_max,close_approach_date_full,orbiting_body,relative_velocity_kilometers_per_second,relative_velocity_kilometers_per_hour,miss_distance_astronomical,miss_distance_lunar,miss_distance_kilometers,date_added,data
0,2329437,329437 (2002 OA22),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2329437,19.41,True,False,348.782733,779.901901,2023-Sep-26 15:59,Earth,15.7156596389,56576.3746998676,0.2974523576,115.7089671064,44498239.123438312,2023-10-04 16:38:22.332234,{'links': {'self': 'http://api.nasa.gov/neo/re...
1,3177195,(2004 EO20),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3177195,22.50,False,False,84.053340,187.948982,2023-Sep-26 09:47,Earth,3.7493697142,13497.730971158,0.2326105867,90.4855182263,34798048.309770329,2023-10-04 16:38:22.333232,{'links': {'self': 'http://api.nasa.gov/neo/re...
2,3459484,(2009 JM1),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3459484,23.20,False,False,60.891262,136.157002,2023-Sep-26 20:54,Earth,18.9668256714,68280.5724171152,0.2440373587,94.9305325343,36507469.061945969,2023-10-04 16:38:22.333232,{'links': {'self': 'http://api.nasa.gov/neo/re...
3,3670290,(2014 HJ184),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3670290,21.65,False,False,124.324001,277.996916,2023-Sep-26 02:59,Earth,19.5415017367,70349.4062519443,0.4265898628,165.9434566292,63816934.838472236,2023-10-04 16:38:22.333232,{'links': {'self': 'http://api.nasa.gov/neo/re...
4,3843427,(2019 OV3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3843427,24.26,False,False,37.372743,83.567994,2023-Sep-26 04:14,Earth,10.2016957272,36726.104618078,0.3800277452,147.8307928828,56851341.222822724,2023-10-04 16:38:22.333232,{'links': {'self': 'http://api.nasa.gov/neo/re...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,3843536,(2019 PM2),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3843536,21.83,False,False,114.433973,255.882143,2023-Sep-30 10:56,Earth,13.887658663,49995.571186647,0.4228055303,164.4713512867,63250806.757100461,2023-10-04 16:38:27.648992,{'links': {'self': 'http://api.nasa.gov/neo/re...
184,3989291,(2020 BD13),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3989291,23.30,False,False,58.150704,130.028927,2023-Sep-30 06:22,Earth,29.6233023399,106643.8884235018,0.4551151295,177.0397853755,68084253.977974165,2023-10-04 16:38:27.648992,{'links': {'self': 'http://api.nasa.gov/neo/re...
185,54051300,(2020 QG3),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54051300,21.32,True,False,144.728805,323.623447,2023-Sep-30 07:07,Earth,13.0569466774,47005.0080385244,0.0572520013,22.2710285057,8564777.447717231,2023-10-04 16:38:27.648992,{'links': {'self': 'http://api.nasa.gov/neo/re...
186,54115824,(2021 CN5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54115824,27.80,False,False,7.320740,16.369672,2023-Sep-30 22:03,Earth,19.726973613,71017.105006692,0.1384929905,53.8737733045,20718256.388730235,2023-10-04 16:38:27.648992,{'links': {'self': 'http://api.nasa.gov/neo/re...


Adding date awareness to allow for updating data without duplicates and loading previous data

In [58]:
datetime.date.today() - datetime.timedelta(days=365)

datetime.date(2022, 10, 4)

In [59]:
try:
    with open('last_executed.txt') as f:
        last_execution_date = datetime.datetime.strptime(f.read(), '%Y-%m-%d').date()
except FileNotFoundError:
        last_execution_date = (datetime.date.today() - datetime.timedelta(days=365))

last_execution_date

datetime.date(2023, 10, 4)

In [60]:
delta = (datetime.date.today() - last_execution_date).days
delta

0

In [61]:
# while delta % 7 != 0:
#     variable = (datetime.date.today() - datetime.timedelta(delta%7))
#     date = variable.strftime('%Y-%m-%d')

#     link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={date}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
    
#     delta = delta - delta % 7
#     print(link)
# while delta // 7 != 0:
#     variable = variable - datetime.timedelta(days=7)
#     date = variable.strftime('%Y-%m-%d')

#     link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={date}&end_date=&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
    
#     delta = delta - 7
#     print(link)

In [62]:
delta = 8

for x in range (0, delta, 7):
    print(x)

0
7


In [63]:
delta = 14
def get_links():
    if delta == 0:
        return []

    links = []

    if delta % 7 != 0:
        end_date = datetime.date.today()
        start_date = (end_date - datetime.timedelta(days=delta%7)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

    if delta == 7:
        end_date = datetime.date.today()
        start_date = (end_date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)
    else:
        for x in range(delta%7, delta, 7):
            end_date = (datetime.date.today() - datetime.timedelta(days=x+1))
            start_date = (end_date - datetime.timedelta(days=6)).strftime('%Y-%m-%d')
            end_date = end_date.strftime('%Y-%m-%d')
            link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
            links.append(link)   

    return links

get_links() 
        

['https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-27&end_date=2023-10-03&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-20&end_date=2023-09-26&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [64]:
delta = 7
def get_links():
    if delta == 0:
        return []

    links = []

    if delta % 7 != 0:
        end_date = datetime.date.today()
        start_date = (end_date - datetime.timedelta(days=delta%7)).strftime('%Y-%m-%d')
        end_date = end_date.strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

        for x in range(delta%7, delta, 7):
            end_date = (datetime.date.today() - datetime.timedelta(days=x+1))
            start_date = (end_date - datetime.timedelta(days=6)).strftime('%Y-%m-%d')
            end_date = end_date.strftime('%Y-%m-%d')
            link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
            links.append(link)   

    if delta % 7 == 0:
        # end_date = datetime.date.today()
        # start_date = (end_date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
        # end_date = end_date.strftime('%Y-%m-%d')
        # link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        # links.append(link)

        for x in range(0, delta, 7):
            end_date = (datetime.date.today() - datetime.timedelta(days=x+1))
            start_date = (end_date - datetime.timedelta(days=7)).strftime('%Y-%m-%d')
            end_date = end_date.strftime('%Y-%m-%d')
            link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
            links.append(link)  
        
    return links

get_links()

['https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-26&end_date=2023-10-03&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [65]:
delta = 15

for x in range (0, delta, 7):
    print(x)

def get_links():
    if delta == 0:
        return []

    links = []

    if delta % 7 != 0:
        end_date = datetime.date.today()
        start_date = (datetime.date.today() - datetime.timedelta(days=delta%7-1)).strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)


    for x in range(delta%7, delta, 7):
        end_date = (datetime.date.today() - datetime.timedelta(days=x)).strftime('%Y-%m-%d')
        start_date = (datetime.date.today() - datetime.timedelta(days=x+6)).strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

    return links

get_links() 

0
7
14


['https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-10-04&end_date=2023-10-04&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-27&end_date=2023-10-03&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-20&end_date=2023-09-26&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [66]:
delta = 14
def get_links(delta):
    if delta == 0:
        return []

    links = []

    def build_link(x, y):
        end_date = x.strftime('%Y-%m-%d')
        start_date = (datetime.date.today() - datetime.timedelta(days=y)).strftime('%Y-%m-%d')
        link = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ'
        links.append(link)

    if delta % 7 != 0:
        build_link(datetime.date.today(), delta%7-1)
    

    for x in range(delta%7, delta, 7):
        build_link((datetime.date.today() - datetime.timedelta(days=x)), x+6)

    return links

get_links(delta) 

['https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-28&end_date=2023-10-04&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ',
 'https://api.nasa.gov/neo/rest/v1/feed?start_date=2023-09-21&end_date=2023-09-27&api_key=Gt87ibmZefPpnhl8gfz5gWWiTuftebq6IgJBFNdQ']

In [77]:
def extract_load(links):
    
    for link in links:
    
        response = requests.get(link)
        data = response.json()
        neo_data = data['near_earth_objects']

        cursor = conn.cursor()

        cursor.executescript('''

            DROP TABLE IF EXISTS neo;                   

            CREATE TABLE IF NOT EXISTS neo (
                            date_added datetime,
                            data json
            )
        ''')

        for date, neo_list in neo_data.items():
            for neo in neo_list:
                cursor.execute('INSERT INTO neo VALUES (?, ?)',
                               (datetime.datetime.today(), json.dumps(neo)))
        conn.commit()

        with open('last_executed.txt', 'w') as f:
            f.write(str(datetime.date.today()))

In [104]:
def transform():
    df = pd.read_sql_query('''SELECT * FROM neo''', conn)

    df['data'] = df['data'].apply(json.loads)

    json_df = pd.json_normalize(df['data'])

    for n in json_df.index.values:
        json_df['close_approach_data'][n] = {k: v for element in json_df['close_approach_data'][n]
                                             for k, v in element.items()}

    cad_df = pd.json_normalize(json_df['close_approach_data'])

    df = pd.concat([json_df, cad_df, df], axis=1)

    df = df.drop(['neo_reference_id', 'close_approach_data', 'links.self',
                  'estimated_diameter.kilometers.estimated_diameter_min',
                  'estimated_diameter.kilometers.estimated_diameter_max',
                  'estimated_diameter.miles.estimated_diameter_min',
                  'estimated_diameter.miles.estimated_diameter_max',
                  'estimated_diameter.feet.estimated_diameter_min',
                  'estimated_diameter.feet.estimated_diameter_max',
                  'close_approach_date', 'epoch_date_close_approach',
                  'relative_velocity.miles_per_hour',
                  'miss_distance.miles'], axis=1
                 )

    df = df.rename(columns={
        'estimated_diameter.meters.estimated_diameter_min': 'estimated_diameter_meters_min',
        'estimated_diameter.meters.estimated_diameter_max': 'estimated_diameter_meters_max',
        'relative_velocity.kilometers_per_second': 'relative_velocity_kilometers_per_second',
        'relative_velocity.kilometers_per_hour': 'relative_velocity_kilometers_per_hour',
        'miss_distance.astronomical': 'miss_distance_astronomical',
        'miss_distance.lunar': 'miss_distance_lunar',
        'miss_distance.kilometers': 'miss_distance_kilometers',
        })

    cols = ['relative_velocity_kilometers_per_second',
            'relative_velocity_kilometers_per_hour',
            'miss_distance_astronomical',
            'miss_distance_lunar',
            'miss_distance_kilometers'
            ]

    df[cols] = df[cols].astype('float')
    
    df['close_approach_date_full'] = df['close_approach_date_full'].astype('datetime64')

    return df

In [113]:
extract_load(get_links(50))
df = transform()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 17 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   id                                       81 non-null     object        
 1   name                                     81 non-null     object        
 2   nasa_jpl_url                             81 non-null     object        
 3   absolute_magnitude_h                     81 non-null     float64       
 4   is_potentially_hazardous_asteroid        81 non-null     bool          
 5   is_sentry_object                         81 non-null     bool          
 6   estimated_diameter_meters_min            81 non-null     float64       
 7   estimated_diameter_meters_max            81 non-null     float64       
 8   close_approach_date_full                 81 non-null     datetime64[ns]
 9   orbiting_body                            81 n

C:\Users\Jack\AppData\Local\Temp\ipykernel_6880\311514158.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  json_df['close_approach_data'][n] = {k: v for element in json_df['close_approach_data'][n]


In [115]:
df[df['close_approach_date_full'].duplicated()==True]

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_meters_min,estimated_diameter_meters_max,close_approach_date_full,orbiting_body,relative_velocity_kilometers_per_second,relative_velocity_kilometers_per_hour,miss_distance_astronomical,miss_distance_lunar,miss_distance_kilometers,date_added,data
79,2613403,613403 (2006 GB),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2613403,20.23,True,False,239.086457,534.613571,2023-08-20 07:19:00,Earth,8.088632,29119.075681,0.128284,49.902339,1.919096e+07,2023-10-04 17:04:18.898798,{'links': {'self': 'http://api.nasa.gov/neo/re...


In [116]:
df[df['close_approach_date_full']=='2023-08-20 07:19:00']

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_meters_min,estimated_diameter_meters_max,close_approach_date_full,orbiting_body,relative_velocity_kilometers_per_second,relative_velocity_kilometers_per_hour,miss_distance_astronomical,miss_distance_lunar,miss_distance_kilometers,date_added,data
72,3329278,(2006 GB),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3329278,20.34,True,False,227.276732,508.206223,2023-08-20 07:19:00,Earth,8.088629,29119.064698,0.128284,49.902283,1.919094e+07,2023-10-04 17:04:18.898798,{'links': {'self': 'http://api.nasa.gov/neo/re...
79,2613403,613403 (2006 GB),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2613403,20.23,True,False,239.086457,534.613571,2023-08-20 07:19:00,Earth,8.088632,29119.075681,0.128284,49.902339,1.919096e+07,2023-10-04 17:04:18.898798,{'links': {'self': 'http://api.nasa.gov/neo/re...


In [118]:
df_hazard = df[df['is_potentially_hazardous_asteroid']==1]
df_hazard_wow = df_hazard[df_hazard['miss_distance_lunar']<50]
df_hazard_wow

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_meters_min,estimated_diameter_meters_max,close_approach_date_full,orbiting_body,relative_velocity_kilometers_per_second,relative_velocity_kilometers_per_hour,miss_distance_astronomical,miss_distance_lunar,miss_distance_kilometers,date_added,data
64,2004769,4769 Castalia (1989 PB),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2004769,17.50,True,False,840.533402,1879.489824,2023-08-22 09:21:00,Earth,18.197448,65510.814546,0.110026,42.800224,1.645970e+07,2023-10-04 17:04:18.898798,{'links': {'self': 'http://api.nasa.gov/neo/re...
72,3329278,(2006 GB),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3329278,20.34,True,False,227.276732,508.206223,2023-08-20 07:19:00,Earth,8.088629,29119.064698,0.128284,49.902283,1.919094e+07,2023-10-04 17:04:18.898798,{'links': {'self': 'http://api.nasa.gov/neo/re...
79,2613403,613403 (2006 GB),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2613403,20.23,True,False,239.086457,534.613571,2023-08-20 07:19:00,Earth,8.088632,29119.075681,0.128284,49.902339,1.919096e+07,2023-10-04 17:04:18.898798,{'links': {'self': 'http://api.nasa.gov/neo/re...


In [122]:
df[df['estimated_diameter_meters_max']>1000]

,id,name,nasa_jpl_url,absolute_magnitude_h,is_potentially_hazardous_asteroid,is_sentry_object,estimated_diameter_meters_min,estimated_diameter_meters_max,close_approach_date_full,orbiting_body,relative_velocity_kilometers_per_second,relative_velocity_kilometers_per_hour,miss_distance_astronomical,miss_distance_lunar,miss_distance_kilometers,date_added,data
22,2351815,351815 (2006 OF15),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2351815,18.38,False,False,560.473362,1253.256538,2023-08-16 04:47:00,Earth,11.389443,41001.995575,0.443900,172.677133,6.640651e+07,2023-10-04 17:04:18.897801,{'links': {'self': 'http://api.nasa.gov/neo/re...
25,3609659,(2012 TT),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3609659,18.02,False,False,661.538275,1479.244553,2023-08-16 19:51:00,Earth,23.208324,83549.966213,0.481909,187.462714,7.209260e+07,2023-10-04 17:04:18.897801,{'links': {'self': 'http://api.nasa.gov/neo/re...
61,54145076,(2021 JN5),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=54145076,18.56,False,False,515.887466,1153.559443,2023-08-21 07:13:00,Earth,28.180813,101450.928229,0.323600,125.880338,4.840985e+07,2023-10-04 17:04:18.898798,{'links': {'self': 'http://api.nasa.gov/neo/re...
64,2004769,4769 Castalia (1989 PB),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=2004769,17.50,True,False,840.533402,1879.489824,2023-08-22 09:21:00,Earth,18.197448,65510.814546,0.110026,42.800224,1.645970e+07,2023-10-04 17:04:18.898798,{'links': {'self': 'http://api.nasa.gov/neo/re...
66,3444275,(2009 AE16),http://ssd.jpl.nasa.gov/sbdb.cgi?sstr=3444275,18.77,True,False,468.333233,1047.224945,2023-08-22 12:05:00,Earth,37.819594,136150.539742,0.281400,109.464508,4.209681e+07,2023-10-04 17:04:18.898798,{'links': {'self': 'http://api.nasa.gov/neo/re...


In [124]:
from bokeh.plotting import figure, show

# prepare some data
x = [1, 2, 3, 4, 5]
y = [6, 7, 2, 4, 5]

# create a new plot with a title and axis labels
p = figure(title="Simple line example", x_axis_label="x", y_axis_label="y")

# add a line renderer with legend and line thickness
p.line(x, y, legend_label="Temp.", line_width=2)

# show the results
show(p)

In [135]:
df_diameter = df[['estimated_diameter_meters_min', 'estimated_diameter_meters_max']]
df_diameter

,estimated_diameter_meters_min,estimated_diameter_meters_max
0,290.104841,648.694146
1,100.127870,223.892723
2,15.295194,34.201092
3,7.320740,16.369672
4,26.336313,58.889786
...,...,...
76,46.190746,103.285648
77,160.160338,358.129403
78,303.777061,679.266159
79,239.086457,534.613571


In [146]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource

# prepare some data
data = df_diameter
source = ColumnDataSource(data)
# create a new plot with a title and axis labels
p = figure()

# add a line renderer with legend and line thickness
p.circle(x='estimated_diameter_meters_min', y='estimated_diameter_meters_max', source=source)

# show the results
show(p)